In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType

In [0]:
name_schema = StructType(fields=[StructField("forename", StringType(), True),
                                 StructField("surname", StringType(), True)
])

In [0]:
driver_schema = StructType(fields=[StructField("driverId", IntegerType(), True),
                                   StructField("driverRef", StringType(), True),
                                   StructField("number", IntegerType(), True),
                                   StructField("code", StringType(), True),
                                   StructField("name", name_schema), 
                                   StructField("dob", DateType(), True),
                                   StructField("nationality", StringType(), True),
                                   StructField("url", StringType(), True),
])

In [0]:
driver_df = spark.read.schema(driver_schema).json("/mnt/covid19projectdatalake/raw/drivers.json")

In [0]:
from pyspark.sql.functions import current_timestamp, concat, lit, col

In [0]:
driver_with_columns_df = driver_df.withColumnRenamed("driverId", "driver_id").withColumnRenamed("driverRef", "driver_ref").withColumn("ingestion_date", current_timestamp()).withColumn("name", concat(col("name.forename"), lit(" "), col("name.surname")))

In [0]:
driver_final_df = driver_with_columns_df.drop(col("url"))

In [0]:
driver_final_df.write.mode("overwrite").parquet("/mnt/covid19projectdatalake/processed/driver")